## Conversion of simulations for analysis in RevBayes

I need to write a file in Rev to run an analysis in RevBayes

The first part imports some libraries

In [ ]:
import sys
import re

This is the template for defining the stochastic node for each fossil in the Rev language

In [ ]:
calibration_templ = """
tmrca_NODENAME := tmrca(timetree,NODENAME)
n_TMRCA_NODENAME := -(tmrca_NODENAME)
NODENAME_fossil_age <- -FOSSILAGE
NODENAME_true_age <- TRUENODEAGE
NODENAME_lambda <- 1.0 / (NODENAME_true_age - FOSSILAGE)
NODENAME_fossil_snode ~ dnExponential(lambda=NODENAME_lambda, offset=n_TMRCA_NODENAME)
NODENAME_fossil_snode.clamp(NODENAME_fossil_age)
"""

Next, I set the number of simulation arguments. Since this is a prototype script, the number of replicates is equal to 1.

In [ ]:
nreps = 1

### Reading from File

This script will read the contents of a template Rev file containing all of the necessary Rev code with placeholders in it. The placeholders are simply text tags that will get replaced with the run-specific code. 

Set the template file name. This is a simple string contained in a variable.

In [ ]:
rev_temp_fn = "rev_temp.Rev"

Open the file and assign it to a variable called "file". To do this, the "open" function is used and the arguments of this function are the file name (a string) and another string that specifies the mode of the file. The most common modes are 'r' or 'w' that stand for "read" and "write", respectively. 

In [ ]:
file = open(rev_temp_fn, 'r')

Now, the entire contents of the file called "rev_temp.Rev" will be read into memory and stored in a variable.

In [ ]:
rev_template = file.read()

Close the file

In [ ]:
file.close()

### Looping Through Each Replicate

Each simulation replicate will be treated in the same way. We use a for loop to iterate over each simulation replicate.  

First we create a string that is the prefix of the filename used by all simulation replicates

In [ ]:
f_prfx = "B_sfbd_"

Here is the entire loop

In [ ]:
for i in range(nreps):
	cal_fn = f_prfx + str(i) + ".calrnd_true.cal"
	dat_fn = f_prfx + str(i) + ".dat"
	tre_fn = f_prfx + str(i) + ".SCL.tre"
	out_pfx = "R" + f_prfx + str(i)
	rev_file_name = out_pfx + "_tru_cal.Rev"
	rev_out = rev_template.replace("SEQDATAFILE",dat_fn)
	rev_out = rev_out.replace("TRUETREEFILE",tre_fn)
	rev_out = rev_out.replace("OUTPUTFILEPFX",out_pfx)
	file = open(cal_fn, 'rU')
	cal_d = file.read().strip().split('\n')
	file.close()
	nodes = {}
	clade_names = []
	c = 0
	root_node = []
	oldest = 0.0
	for line in cal_d[1:]:
		l = line.split()
		if(l[1] == 'root'):
			min_age = l[-3]
			tru_age = l[-1]
			root_node = [min_age, tru_age]
			if(float(min_age) > oldest):
				oldest = float(min_age)
		else:
			taxa = l[1:3]
			min_age = l[-3]
			if(float(min_age) > oldest):
				oldest = float(min_age)
			tru_age = l[-1]
			clade = "clade" + str(c)
			clade_names += [clade]
			tax_str = clade + " <- clade(\"" + taxa[0] + "\",\"" + taxa[1] + "\")"
			nodes[clade] = [tax_str, min_age, tru_age]
			c += 1
	constraints_def = ""
	constraints_names = ""
	for j in clade_names:
		constraints_def += nodes[j][0] + "\n"
		constraints_names += j + ", "
	rev_out = rev_out.replace("CLADEVARDEF",constraints_def)
	rev_out = rev_out.replace("ROOTAGEMIN",str(oldest))
	calibration_defs = ""
	for j in clade_names:
		calib = calibration_templ.replace("NODENAME",j)
		foss_age = nodes[j][1]
		true_age = nodes[j][2]
		calib = calib.replace("FOSSILAGE",foss_age)
		calib = calib.replace("TRUENODEAGE",true_age)
		calibration_defs += calib + "\n"
	rev_out = rev_out.replace("CALIBRATIONDEFS",calibration_defs)
	out = open(rev_file_name,'w')
	out.write(rev_out)
	out.close()